<center><h2>LSTM's In the Market</h2></center>
<center><h4>By: Adam Lieberman</h4></center>

<p>Let us create a Long Short Term Memory (LSTM) model to predict whether a stock will go up or down:</p>
<p><b>Disclaimer:</b> This is not financial advice. I am not responsible for any use, modification, or any attempts based off of this article or code. This is simply a demonstration of how deep learning could be applied in finance.</p>
<br>
<p>Here our are imports:</p>

In [54]:
import numpy as np

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers import LSTM

from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

import pandas_datareader.data as web

<p>Let us obtain some stock data from Yahoo. We will obtain training data from 1999-01-01 to 2017-01-01:</p>

In [5]:
stock = "AAPL"
start = '1999-01-01'
end = '2017-01-01'
df_AAPL = web.DataReader(stock,"yahoo",start,end)
print(df_AAPL)

                  Open        High         Low       Close     Volume  \
Date                                                                    
1999-01-04   42.125000   42.249999   40.000001   41.250000  238221200   
1999-01-05   41.937499   43.937501   41.499999   43.312500  352528400   
1999-01-06   44.124999   44.124999   41.000000   41.750001  337142400   
1999-01-07   42.249999   45.062500   42.125000   44.999999  357254800   
1999-01-08   46.562501   46.875001   43.999999   44.999999  169708000   
1999-01-11   45.749999   46.062500   44.874999   45.874999  140243600   
1999-01-12   46.312499   46.624999   44.124999   46.125001  205184000   
1999-01-13   42.875000   47.312501   42.249999   46.500000  261954000   
1999-01-14   45.500000   46.000001   41.062499   41.374999  430964800   
1999-01-15   41.812499   42.125000   40.000001   41.312501  251501600   
1999-01-19   41.937499   42.312500   40.375000   40.875001  133722400   
1999-01-20   41.062499   42.000000   40.499999   40

<p>We will let our 6 simple features be Open, High, Low, Close, Volume, and Adj Close. Let us now generate some labels for our data. We will look at the adjusted close column and examine the next day's adjusted close price. If the next day's price is higher we will mark a 1, if the next day's price is lower we will mark a 0. For instance</p>

In [6]:
print(df_AAPL["Adj Close"][0:4])

Date
1999-01-04    1.336083
1999-01-05    1.402887
1999-01-06    1.352278
1999-01-07    1.457545
Name: Adj Close, dtype: float64


<p>On 01-05 we see that the price is higher than on 01-04 so that means it would have been goot to buy the stock on 01-04. Thus, we will let the label for 1999-01-04 be a 1. On 01-06 the price was lower than on 01-05 so we will let the 1999-01-05 label be a 0. We create our lables as follows:</p>

In [20]:
dollar_change = df_AAPL["Adj Close"] - df_AAPL["Adj Close"].shift(-1)
dollar_change = dollar_change.apply(lambda i: 0 if float(i) >= 0 else 1 )
labels = dollar_change.values
labels = labels[0:-1]
print(labels)

[1 0 1 ..., 0 0 0]


<p>We now need to normalize our features and drop the last row of our df_AAPL dataframe:</p>

In [10]:
#Normalize Features
df_AAPL_norm =(df_AAPL-df_AAPL.mean())/df_AAPL.std()

#Drop last row because of NaN
df_AAPL_norm = df_AAPL_norm[0:-1]

print(df_AAPL_norm)

                Open      High       Low     Close    Volume  Adj Close
Date                                                                   
1999-01-04 -0.715209 -0.719726 -0.722374 -0.720440  1.084597  -0.850798
1999-01-05 -0.716307 -0.709921 -0.713505 -0.708352  2.233389  -0.849038
1999-01-06 -0.703496 -0.708832 -0.716461 -0.717510  2.078759  -0.850371
1999-01-07 -0.714476 -0.703384 -0.709810 -0.698463  2.280890  -0.847598
1999-01-08 -0.689221 -0.692853 -0.698723 -0.698463  0.396037  -0.847598
1999-01-11 -0.693979 -0.697574 -0.693549 -0.693335  0.099919  -0.846852
1999-01-12 -0.690685 -0.694305 -0.697984 -0.691870  0.752572  -0.846639
1999-01-13 -0.710816 -0.690311 -0.709071 -0.689672  1.323113  -0.846319
1999-01-14 -0.695443 -0.697937 -0.716092 -0.719707  3.021678  -0.850691
1999-01-15 -0.717039 -0.720453 -0.722374 -0.720074  1.218066  -0.850745
1999-01-19 -0.716307 -0.719363 -0.720157 -0.722638  0.034380  -0.851118
1999-01-20 -0.721431 -0.721179 -0.719418 -0.724469  0.645499  -0

<p>We now obtain a numpy array from our df_AAPL_norm dataframe that we can use as training data:</p>

In [12]:
x_features = df_AAPL_norm.values
print(x_features)

[[-0.71520852 -0.71972627 -0.72237425 -0.72043982  1.0845975  -0.85079785]
 [-0.7163066  -0.7099211  -0.7135051  -0.70835244  2.2333894  -0.84903818]
 [-0.70349571 -0.70883165 -0.71646148 -0.71750954  2.07875949 -0.85037126]
 ..., 
 [-0.27952126 -0.2807456  -0.27010609 -0.27497963 -1.12565046  2.18939852]
 [-0.27366486 -0.27946733 -0.27182081 -0.2779099  -1.0994299   2.17628498]
 [-0.27993121 -0.28475483 -0.27063822 -0.27808571 -1.15838746  2.17549821]]


<p>We have a binary classification problem as we want to classify a stock as 0 or 1. Where 0 means the stock is going down the next day and 1 means the stock is going up the next day. To do so we will create a 1-hot encoded vecor:</p>

In [23]:
x_labels = np.zeros((len(x_features),2))
c = 0
for i in labels:
    
    #This means buy: [0,1]
    if i == 1:
        x_labels[c][1] = 1
        
    #This means sell [1,0]
    else: 
        x_labels[c][0] = 1
    c = c +1 

print(x_labels)

[[ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 ..., 
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]


<p>We now need to reshape our features and labels for the LSTM:</p>

In [26]:
#reshaping for LSTM 
num_samples = len(x_features)
num_classes = len(x_labels[0])
num_features = len(x_features[0])

#reshape labels
x_labels = x_labels.reshape(num_samples,1,num_classes)

#reshape features
x_features = x_features.reshape(num_samples,1,num_features)

<p>Let us now set some parameters for our LSTM:</p>

In [28]:
shape = x_features.shape[2]
print(shape)
num_classes = len(x_labels[0][0])
print(num_classes)

6
2


<p>Let us now create a function to build our model:</p>

In [29]:
def create_model(shape,num_classes):
    model = Sequential()
    model.add(LSTM(32, input_shape=(None,shape), return_sequences=True))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['mse'])
    return model

<p>Let us now train our model:</p>

In [31]:
model = create_model(shape,num_classes)
history = model.fit(x_features, x_labels,
              batch_size=100, epochs=200,
              verbose = 1)

Epoch 1/200
4528/4528 [==============================] - 1s - loss: 0.6934 - mean_squared_error: 0.2501      
Epoch 2/200
4528/4528 [==

<p>Let us now create some test data, we will use the above training process with different dates to do so:</p>

In [40]:
start = '2017-01-02'
end = '2017-04-25'
sample = web.DataReader('AAPL',"yahoo",start,end)

dollar_change = sample["Adj Close"] - sample["Adj Close"].shift(-1)
dollar_change = dollar_change.apply(lambda i: 0 if float(i) >= 0 else 1 )
labels = dollar_change.values
labels = labels[0:-1]

sample =(sample-sample.mean())/sample.std()
sample = sample.values
sample = sample.reshape(sample.shape[0],1,num_features)

sample =(sample-sample.mean())/sample.std()

sample = sample[0:-1]

<p>We can now predict our samples on our model:</p>

In [52]:
preds = model.predict_classes(sample)
preds = preds.flatten()
print("\n",preds)

32/75 [===========>..................] - ETA: 0s
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1]


<p>Let us now evaluate our accuracy:</p>

In [53]:
acc = accuracy_score(labels,preds)
print(acc)

0.52


<p>We see that our model was accurate 52% of the time. We note that we did not use intersting features or heavily tune our model. Doing so, we could obtain better accuracy.</p>